### Import libraries and parameters
First, we are going to import the libraries needed to perform data ingestion.

In [2]:
# Import libraries
import os
import urllib.request
import datetime
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from dateutil import parser
import zipfile
from collections import Counter

from modules.config import PROCESSED_DATA_DIR, RAW_DATA_DIR, INTERIM_DATA_DIR, URL_LIST, POLLUTANTS, MUNICIPALITY

2024-10-03 10:03:46.037 | INFO     | modules.config:<module>:11 - PROJ_ROOT path is: /mnt/Data/Documents/Unison/1 Semestre/Ingeniería de Características/Projects/Sonora-River-Farming/Data-Science-Project


### Download data
We need to specify our working directory and be sure not to download data repeatedly unnecessarily.

In [3]:
# Working Directory
print(os.getcwd())

# First we're going to work with the Conagua water quality database
WATER_QUALITY_SOURCE = "https://files.conagua.gob.mx/aguasnacionales/TODOS%20LOS%20MONITOREOS.xlsb"
WATER_QUALITY_FILE = "water_quality_data.xlsb"
SUBDIR = "../data/raw/"

/mnt/Data/Documents/Unison/1 Semestre/Ingeniería de Características/Projects/Sonora-River-Farming/Data-Science-Project/notebooks


If the file doesn't exist, it will be downloaded and a text file will be generated with a brief description of the data source, 
the download date and the data download link.

In [4]:
if not os.path.exists(SUBDIR + WATER_QUALITY_FILE):
    if not os.path.exists(SUBDIR):
        os.makedirs(SUBDIR)
    urllib.request.urlretrieve(WATER_QUALITY_SOURCE, SUBDIR + WATER_QUALITY_FILE)  

    with open(SUBDIR + "info.txt", 'w') as f:
        f.write("Information from water quality monitoring sites operated by Conagua throughout the country\n\n")
        info = (
            "The information includes data on lotic, lentic, coastal, and underground water bodies, covering physicochemical "
            "and microbiological parameters according to the type of water body. These data are organized in an Excel file "
            "with three spreadsheets.\n\n"

            "First sheet: Contains details about the monitoring sites, such as key, name, aquifer, state, municipality, type "
            "of water body, latitude, longitude, among others.\n\n"

            "Second sheet: Presents the results of the monitoring, grouped by site, type of water body, date of completion, "
            "and the physicochemical and microbiological parameters recorded.\n\n"

            "Third sheet: Offers a dictionary that describes each parameter, indicating its key, name, and unit of measurement.\n\n"

            "The data was obtained from the National Water Commission (https://www.gob.mx/conagua/articulos/calidad-del-agua) "
            "dated August 6, 2024."
        )
        f.write(info + '\n')
        f.write("Downloaded on " + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") + "\n")
        f.write("From: " + WATER_QUALITY_SOURCE + "\n")
        f.write("Name: " + WATER_QUALITY_FILE + "\n")

Using pandas read_excel function to read an Excel file, the contents of "WATER_QUALITY_FILE" is stored in the DataFrame df_water

In [5]:
df_water =  pd.read_excel(SUBDIR + WATER_QUALITY_FILE, sheet_name=None)

print(f"\nSheet number: {len(df_water)}")
print(f"\nSheet name: { ', '.join(df_water.keys())}")


Sheet number: 3

Sheet name: Sitios, Resultados, Simbología


In [6]:
df_water_site = pd.read_excel(SUBDIR + WATER_QUALITY_FILE, sheet_name=0)

We show basic information about the data in the DataFrame "df_water_site" to understand its structure a little.

In [7]:
df_water_site.head()

,CLAVE SITIO,NOMBRE DEL SITIO,CUENCA,CLAVE ACUÍFERO,ACUÍFERO,ORGANISMO CUENCA,DIRECCIÓN LOCAL,ESTADO,MUNICIPIO,CUERPO DE AGUA,TIPO DE CUERPO DE AGUA,SUBTIPO CUERPO AGUA,LATITUD,LONGITUD
0,OCPYU4929,COSGAYA,NaN,3105.0,PENÍNSULA DE YUCATÁN,PENÍNSULA DE YUCATÁN,NaN,YUCATÁN,MÉRIDA,ACUÍFERO PENÍNSULA DE YUCATÁN,SUBTERRÁNEO (ESTUDIO ESPECIAL),POZO,21.096990,-89.704600
1,OCPYU4946,POZO 1 DEL SISTEMA DE AGUA POTABLE DE SOTUTA,NaN,3105.0,PENÍNSULA DE YUCATÁN,PENÍNSULA DE YUCATÁN,NaN,YUCATÁN,SOTUTA,ACUÍFERO PENÍNSULA DE YUCATÁN,SUBTERRÁNEO (ESTUDIO ESPECIAL),POZO,20.597040,-89.007570
2,OCPYU4953W1,ANILLO DE CENOTES DE YUCATAN 5,NaN,3105.0,PENÍNSULA DE YUCATÁN,PENÍNSULA DE YUCATÁN,NaN,YUCATÁN,KOPOMÁ,ACUÍFERO PENÍNSULA DE YUCATÁN,SUBTERRÁNEO (ESTUDIO ESPECIAL),CENOTE,20.689430,-89.876050
3,OCPYU6396,CACALCHEN,NaN,3105.0,PENÍNSULA DE YUCATÁN,PENÍNSULA DE YUCATÁN,NaN,YUCATÁN,CACALCHEN,ACUÍFERO PENINSULA DE YUCATAN,SUBTERRÁNEO (ESTUDIO ESPECIAL),POZO,20.982222,-89.227778
4,OCPYU6397,CANSAHCAB,NaN,3105.0,PENÍNSULA DE YUCATÁN,PENÍNSULA DE YUCATÁN,NaN,YUCATÁN,CANSAHCAB,ACUÍFERO PENINSULA DE YUCATAN,SUBTERRÁNEO (ESTUDIO ESPECIAL),POZO,21.156850,-89.089960


In [8]:
print(df_water_site.info())  # Structure summary
print('\n----------------------------------------------------\n')
print(df_water_site.describe())  # Statistical summary of numerical columns
print('\n----------------------------------------------------\n')
print(df_water_site.isnull().sum())  # Check for null values
print('\n----------------------------------------------------\n')
print(df_water_site.duplicated().sum())  # Check for duplicates

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7251 entries, 0 to 7250
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   CLAVE SITIO             7251 non-null   object 
 1   NOMBRE DEL SITIO        7251 non-null   object 
 2   CUENCA                  4625 non-null   object 
 3   CLAVE ACUÍFERO          2623 non-null   float64
 4   ACUÍFERO                2623 non-null   object 
 5   ORGANISMO CUENCA        7251 non-null   object 
 6   DIRECCIÓN LOCAL         3543 non-null   object 
 7   ESTADO                  7251 non-null   object 
 8   MUNICIPIO               7251 non-null   object 
 9   CUERPO DE AGUA          7245 non-null   object 
 10  TIPO DE CUERPO DE AGUA  7251 non-null   object 
 11  SUBTIPO CUERPO AGUA     7251 non-null   object 
 12  LATITUD                 7251 non-null   float64
 13  LONGITUD                7251 non-null   float64
dtypes: float64(3), object(11)
memory usage: 

In [9]:
df_water_dic = pd.read_excel(SUBDIR + WATER_QUALITY_FILE, sheet_name=2)

In [10]:
df_water_result = pd.read_excel(SUBDIR + WATER_QUALITY_FILE, sheet_name=1)

We show basic information about the data in the DataFrame "df_water_result" to understand its structure a little.

In [11]:
df_water_result.head()

,CLAVE SITIO,CLAVE MONITOREO,NOMBRE DEL SITIO,TIPO CUERPO DE AGUA,FECHA REALIZACIÓN,Año,ALC_FEN,ALC_TOT,CO3,HCO3,...,STV,TALIO_TOT,TER-BUTILBENCENO,Ti_TOT,TRANS-CLORDANO,FORMALDEHIDO,CLORATOS,ACID_CLOROACE,ACID_DICLOROACE,ACID_TRICLOROACE
0,BROTE CARMINA 3,BROTE CARMINA 3-131122,CARMINA 3 BROTE,SUBTERRÁNEO,44873,2022,NaN,139.7,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CARMINA 2,CARMINA 2-131122,CARMINA 2,LÓTICO,44873,2022,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CAZEPA-1,CAZEPA-1-260921,POZO SAN FERNANDO 1,SUBTERRÁNEO,44462,2021,NaN,148.645,0,148.645,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CAZEPA-1,CAZEPA-1-310722,POZO SAN FERNANDO 1,SUBTERRÁNEO,44769,2022,NaN,141.3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CAZEPA-1,CAZEPA-1-181023,POZO SAN FERNANDO 1,SUBTERRÁNEO,45217,2023,0,210.24,0,210.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
print(df_water_result.info())  # Structure summary
print('\n----------------------------------------------------\n')
print(df_water_result.describe())  # Statistical summary of numerical columns
print('\n----------------------------------------------------\n')
print(df_water_result.isnull().sum())  # Check for null values
print('\n----------------------------------------------------\n')
print(df_water_result.duplicated().sum())  # Check for duplicates

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127424 entries, 0 to 127423
Columns: 452 entries, CLAVE SITIO to ACID_TRICLOROACE
dtypes: float64(73), int64(1), object(378)
memory usage: 439.4+ MB
None

----------------------------------------------------

                 Año  CONDUC_CAMPO_SUP      pH_CAMPO  pH_CAMPO_SUP  \
count  127424.000000      41627.000000  84457.000000  40916.000000   
mean     2015.911273      30362.139897      7.758606      8.120433   
std         2.422348      26813.992809      0.538180      0.523405   
min      2012.000000          8.680000      2.900000      0.200000   
25%      2014.000000        814.500000      7.400000      7.900000   
50%      2016.000000      37350.000000      7.800000      8.120000   
75%      2018.000000      53748.500000      8.100000      8.400000   
max      2023.000000     556020.000000     13.660000     11.000000   

       pH_CAMPO_MED  pH_CAMPO_FON  H_HELMINTOS  POT_REDOX_CAMPO  \
count  19478.000000  17137.000000   147.000

### Data Frame Merged

Merging df_water_site and df_water_result DataFrames using 'CLAVE SITIO' as the key. The 'inner' join returns only rows with matching values in both DataFrames.

In [13]:
df_water_merged = pd.merge(df_water_site, df_water_result, on='CLAVE SITIO', how='inner')
df_water_merged.head(10)

,CLAVE SITIO,NOMBRE DEL SITIO_x,CUENCA,CLAVE ACUÍFERO,ACUÍFERO,ORGANISMO CUENCA,DIRECCIÓN LOCAL,ESTADO,MUNICIPIO,CUERPO DE AGUA,...,STV,TALIO_TOT,TER-BUTILBENCENO,Ti_TOT,TRANS-CLORDANO,FORMALDEHIDO,CLORATOS,ACID_CLOROACE,ACID_DICLOROACE,ACID_TRICLOROACE
0,OCPYU4929,COSGAYA,NaN,3105.0,PENÍNSULA DE YUCATÁN,PENÍNSULA DE YUCATÁN,NaN,YUCATÁN,MÉRIDA,ACUÍFERO PENÍNSULA DE YUCATÁN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,OCPYU4929,COSGAYA,NaN,3105.0,PENÍNSULA DE YUCATÁN,PENÍNSULA DE YUCATÁN,NaN,YUCATÁN,MÉRIDA,ACUÍFERO PENÍNSULA DE YUCATÁN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,OCPYU4929,COSGAYA,NaN,3105.0,PENÍNSULA DE YUCATÁN,PENÍNSULA DE YUCATÁN,NaN,YUCATÁN,MÉRIDA,ACUÍFERO PENÍNSULA DE YUCATÁN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,OCPYU4929,COSGAYA,NaN,3105.0,PENÍNSULA DE YUCATÁN,PENÍNSULA DE YUCATÁN,NaN,YUCATÁN,MÉRIDA,ACUÍFERO PENÍNSULA DE YUCATÁN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,OCPYU4929,COSGAYA,NaN,3105.0,PENÍNSULA DE YUCATÁN,PENÍNSULA DE YUCATÁN,NaN,YUCATÁN,MÉRIDA,ACUÍFERO PENÍNSULA DE YUCATÁN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,OCPYU4929,COSGAYA,NaN,3105.0,PENÍNSULA DE YUCATÁN,PENÍNSULA DE YUCATÁN,NaN,YUCATÁN,MÉRIDA,ACUÍFERO PENÍNSULA DE YUCATÁN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,OCPYU4929,COSGAYA,NaN,3105.0,PENÍNSULA DE YUCATÁN,PENÍNSULA DE YUCATÁN,NaN,YUCATÁN,MÉRIDA,ACUÍFERO PENÍNSULA DE YUCATÁN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,OCPYU4929,COSGAYA,NaN,3105.0,PENÍNSULA DE YUCATÁN,PENÍNSULA DE YUCATÁN,NaN,YUCATÁN,MÉRIDA,ACUÍFERO PENÍNSULA DE YUCATÁN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,OCPYU4929,COSGAYA,NaN,3105.0,PENÍNSULA DE YUCATÁN,PENÍNSULA DE YUCATÁN,NaN,YUCATÁN,MÉRIDA,ACUÍFERO PENÍNSULA DE YUCATÁN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,OCPYU4929,COSGAYA,NaN,3105.0,PENÍNSULA DE YUCATÁN,PENÍNSULA DE YUCATÁN,NaN,YUCATÁN,MÉRIDA,ACUÍFERO PENÍNSULA DE YUCATÁN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
print(df_water_merged.info())  # Structure summary
print('\n----------------------------------------------------\n')
print(df_water_merged.describe())  # Statistical summary of numerical columns
print('\n----------------------------------------------------\n')
print(df_water_merged.isnull().sum())  # Check for null values
print('\n----------------------------------------------------\n')
print(df_water_merged.duplicated().sum())  # Check for duplicates

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127424 entries, 0 to 127423
Columns: 465 entries, CLAVE SITIO to ACID_TRICLOROACE
dtypes: float64(76), int64(1), object(388)
memory usage: 452.1+ MB
None

----------------------------------------------------

       CLAVE ACUÍFERO        LATITUD       LONGITUD            Año  \
count    13450.000000  127424.000000  127424.000000  127424.000000   
mean      1713.954796      21.391787    -100.672892    2015.911273   
std        985.009996       4.043795       6.134039       2.422348   
min        101.000000      14.534910    -117.124028    2012.000000   
25%       1002.000000      18.507650    -104.329408    2014.000000   
50%       1506.000000      20.377210    -100.002660    2016.000000   
75%       2626.000000      23.998210     -97.074460    2018.000000   
max       3231.000000      32.706497     100.859540    2023.000000   

       CONDUC_CAMPO_SUP      pH_CAMPO  pH_CAMPO_SUP  pH_CAMPO_MED  \
count      41627.000000  84457.000000  40

Select the columns required for the study

In [15]:
df_water_merged_filtered = df_water_merged[['CLAVE SITIO', 'ESTADO', 'MUNICIPIO', 'CUERPO DE AGUA', 'TIPO CUERPO DE AGUA', 'SUBTIPO CUERPO AGUA', 'LATITUD', 'LONGITUD'] + POLLUTANTS]
df_water_merged_filtered.head()

,CLAVE SITIO,ESTADO,MUNICIPIO,CUERPO DE AGUA,TIPO CUERPO DE AGUA,SUBTIPO CUERPO AGUA,LATITUD,LONGITUD,OD_mg/L,DBO_TOT,DQO_TOT,COLI_FEC,E_COLI,N_TOT,P_TOT,TOX_D_48_UT,TOX_FIS_SUP_15_UT
0,OCPYU4929,YUCATÁN,MÉRIDA,ACUÍFERO PENÍNSULA DE YUCATÁN,SUBTERRÁNEO (ESTUDIO ESPECIAL),POZO,21.09699,-89.7046,NaN,NaN,NaN,<3,NaN,6.0318,<0.0014,NaN,NaN
1,OCPYU4929,YUCATÁN,MÉRIDA,ACUÍFERO PENÍNSULA DE YUCATÁN,SUBTERRÁNEO (ESTUDIO ESPECIAL),POZO,21.09699,-89.7046,NaN,NaN,NaN,<2,NaN,4.3016,0.3995,NaN,NaN
2,OCPYU4929,YUCATÁN,MÉRIDA,ACUÍFERO PENÍNSULA DE YUCATÁN,SUBTERRÁNEO (ESTUDIO ESPECIAL),POZO,21.09699,-89.7046,NaN,NaN,NaN,7,NaN,5.114,<0.0014,NaN,NaN
3,OCPYU4929,YUCATÁN,MÉRIDA,ACUÍFERO PENÍNSULA DE YUCATÁN,SUBTERRÁNEO (ESTUDIO ESPECIAL),POZO,21.09699,-89.7046,NaN,NaN,NaN,<3,NaN,2.7859,0.1007,NaN,NaN
4,OCPYU4929,YUCATÁN,MÉRIDA,ACUÍFERO PENÍNSULA DE YUCATÁN,SUBTERRÁNEO (ESTUDIO ESPECIAL),POZO,21.09699,-89.7046,NaN,NaN,NaN,<3,NaN,3.0509,0.0368,NaN,NaN


Filter the DataFrame to obtain only the records of the water bodies that are not "COASTAL" of the affected municipalities of the state of Sonora.

In [16]:
df_water_merged_sonora = df_water_merged_filtered[
    (df_water_merged['ESTADO'] == 'SONORA') &
    (df_water_merged['MUNICIPIO'].isin(MUNICIPALITY))   
]
df_water_merged_sonora = df_water_merged_sonora[~df_water_merged_sonora['TIPO CUERPO DE AGUA'].str.contains('COSTERO', na=False)]
df_water_merged_sonora.head()

,CLAVE SITIO,ESTADO,MUNICIPIO,CUERPO DE AGUA,TIPO CUERPO DE AGUA,SUBTIPO CUERPO AGUA,LATITUD,LONGITUD,OD_mg/L,DBO_TOT,DQO_TOT,COLI_FEC,E_COLI,N_TOT,P_TOT,TOX_D_48_UT,TOX_FIS_SUP_15_UT
94719,OCNOR3987,SONORA,CANANEA,RIO SONORA,LÓTICO,RÍO,30.95804,-110.18906,7.19,3.41,17.5392,145,20,2.2916,0.0617,<1,NaN
94720,OCNOR3987,SONORA,CANANEA,RIO SONORA,LÓTICO,RÍO,30.95804,-110.18906,8.01,3.05,<10,<3,<3,0.7498,0.0931,<1,NaN
94721,OCNOR3987,SONORA,CANANEA,RIO SONORA,LÓTICO,RÍO,30.95804,-110.18906,9.24,6.7,51.084,110,10,0.7643,0.1029,<1,NaN
94722,OCNOR3987,SONORA,CANANEA,RIO SONORA,LÓTICO,RÍO,30.95804,-110.18906,7.07,2.97,<10,110,31,0.9269,0.1127,<1,NaN
94723,OCNOR3987,SONORA,CANANEA,RIO SONORA,LÓTICO,RÍO,30.95804,-110.18906,6.33,2.83,22.736,3448,110,1.3238,0.1331,<1,NaN


In [17]:
print(df_water_merged_sonora.info())  # Structure summary
print('\n----------------------------------------------------\n')
print(df_water_merged_sonora.describe())  # Statistical summary of numerical columns
print('\n----------------------------------------------------\n')
print(df_water_merged_sonora.isnull().sum())  # Check for null values
print('\n----------------------------------------------------\n')
print(df_water_merged_sonora.duplicated().sum())  # Check for duplicates

<class 'pandas.core.frame.DataFrame'>
Index: 473 entries, 94719 to 126167
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CLAVE SITIO          473 non-null    object 
 1   ESTADO               473 non-null    object 
 2   MUNICIPIO            473 non-null    object 
 3   CUERPO DE AGUA       473 non-null    object 
 4   TIPO CUERPO DE AGUA  473 non-null    object 
 5   SUBTIPO CUERPO AGUA  473 non-null    object 
 6   LATITUD              473 non-null    float64
 7   LONGITUD             473 non-null    float64
 8   OD_mg/L              359 non-null    object 
 9   DBO_TOT              349 non-null    object 
 10  DQO_TOT              347 non-null    object 
 11  COLI_FEC             452 non-null    object 
 12  E_COLI               349 non-null    object 
 13  N_TOT                420 non-null    object 
 14  P_TOT                416 non-null    object 
 15  TOX_D_48_UT          350 non-null    o

Clean the columns of contaminants by removing the > and < symbols and converting them to numeric

In [18]:
# Clean the columns of contaminants by removing the > and < symbols and converting them to numeric
for column in POLLUTANTS:
    if column in df_water_merged_sonora.columns:
        df_water_merged_sonora[column] = df_water_merged_sonora[column].astype(str)
        df_water_merged_sonora[column] = (
            df_water_merged_sonora[column]
            .str.replace('<', '', regex=False)
            .str.replace('>', '', regex=False)
        )
        df_water_merged_sonora[column] = pd.to_numeric(df_water_merged_sonora[column], errors='coerce')

print(df_water_merged_sonora.info()) 

<class 'pandas.core.frame.DataFrame'>
Index: 473 entries, 94719 to 126167
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CLAVE SITIO          473 non-null    object 
 1   ESTADO               473 non-null    object 
 2   MUNICIPIO            473 non-null    object 
 3   CUERPO DE AGUA       473 non-null    object 
 4   TIPO CUERPO DE AGUA  473 non-null    object 
 5   SUBTIPO CUERPO AGUA  473 non-null    object 
 6   LATITUD              473 non-null    float64
 7   LONGITUD             473 non-null    float64
 8   OD_mg/L              359 non-null    float64
 9   DBO_TOT              349 non-null    float64
 10  DQO_TOT              347 non-null    float64
 11  COLI_FEC             452 non-null    float64
 12  E_COLI               349 non-null    float64
 13  N_TOT                420 non-null    float64
 14  P_TOT                416 non-null    float64
 15  TOX_D_48_UT          350 non-null    f

In [ ]:
INTERIM_WATER_DATA_DIR = INTERIM_DATA_DIR / 'water_interim.parquet'
table = pa.Table.from_pandas(df_water_merged)

with pq.ParquetWriter(INTERIM_WATER_DATA_DIR, table.schema) as writer:
    writer.write_table(table)

-------------------------------------------------------------------------------
-------------------------------------------------------------------------------

### Another tests

In [19]:
df_water_site_keys = df_water_site.keys()
df_water_site_keys

Index(['CLAVE SITIO', 'NOMBRE DEL SITIO', 'CUENCA', 'CLAVE ACUÍFERO',
       'ACUÍFERO', 'ORGANISMO CUENCA', 'DIRECCIÓN LOCAL', 'ESTADO',
       'MUNICIPIO', 'CUERPO DE AGUA', 'TIPO DE CUERPO DE AGUA',
       'SUBTIPO CUERPO AGUA', 'LATITUD', 'LONGITUD'],
      dtype='object')

In [20]:
df_water_result_keys = df_water_result.keys()
print(df_water_result_keys)

Index(['CLAVE SITIO', 'CLAVE MONITOREO', 'NOMBRE DEL SITIO',
       'TIPO CUERPO DE AGUA', 'FECHA REALIZACIÓN', 'Año', 'ALC_FEN', 'ALC_TOT',
       'CO3', 'HCO3',
       ...
       'STV', 'TALIO_TOT', 'TER-BUTILBENCENO', 'Ti_TOT', 'TRANS-CLORDANO',
       'FORMALDEHIDO', 'CLORATOS', 'ACID_CLOROACE', 'ACID_DICLOROACE',
       'ACID_TRICLOROACE'],
      dtype='object', length=452)


In [21]:
check = df_water_result_keys.isin(['OD_mg/L', 'DBO_TOT', 'DQO_TOT', 'COLI_FEC', 'E_COLI', 'N_TOT', 'P_TOT', 'TOX_D_48_UT', 'TOX_FIS_SUP_15_UT' ])
# print(check)
print(Counter(check))

Counter({np.False_: 443, np.True_: 9})


In [22]:
# Filter 'df_water_site' to only include necesary columns
df_water_site_filtered = df_water_site[['CLAVE SITIO', 'ESTADO', 'MUNICIPIO', 'CUERPO DE AGUA', 'SUBTIPO CUERPO AGUA', 'LATITUD', 'LONGITUD']]
df_water_site_filtered.head()

,CLAVE SITIO,ESTADO,MUNICIPIO,CUERPO DE AGUA,SUBTIPO CUERPO AGUA,LATITUD,LONGITUD
0,OCPYU4929,YUCATÁN,MÉRIDA,ACUÍFERO PENÍNSULA DE YUCATÁN,POZO,21.096990,-89.704600
1,OCPYU4946,YUCATÁN,SOTUTA,ACUÍFERO PENÍNSULA DE YUCATÁN,POZO,20.597040,-89.007570
2,OCPYU4953W1,YUCATÁN,KOPOMÁ,ACUÍFERO PENÍNSULA DE YUCATÁN,CENOTE,20.689430,-89.876050
3,OCPYU6396,YUCATÁN,CACALCHEN,ACUÍFERO PENINSULA DE YUCATAN,POZO,20.982222,-89.227778
4,OCPYU6397,YUCATÁN,CANSAHCAB,ACUÍFERO PENINSULA DE YUCATAN,POZO,21.156850,-89.089960


In [23]:
# The 'municipality' list includes only the municipalities affected by the spill.
municipality = ['ARIZPE', 'BANÁMICHI', 'HUÉPAC', 'ACONCHI', 'SAN FELIPE', 'BAVIÁCORA Y URES']

In [24]:
df_water_site_filtered_sonora = df_water_site_filtered[
    df_water_site_filtered['ESTADO'] == 'SONORA'
]
df_water_site_filtered_sonora['MUNICIPIO'].unique()

array(['AGUA PRIETA', 'CANANEA', 'ACONCHI', 'HUÁSABAS', 'SANTA CRUZ',
       'NOGALES', 'TUBUTAMA', 'ATIL', 'OQUITOA', 'ALTAR', 'TRINCHERAS',
       'SANTA ANA', 'CUMPAS', 'SAHUARIPA', 'CARBÓ',
       'SAN MIGUEL DE HORCASITAS', 'URES', 'BAVIÁCORA', 'BANÁMICHI',
       'ARIZPE', 'BACOACHI', 'IMURIS', 'BENJAMÍN HILL', 'OPODEPE',
       'HERMOSILLO', 'LA COLORADA', 'SOYOPA', 'ONAVAS',
       'SAN PEDRO DE LA CUEVA', 'BACANORA', 'TEPACHE', 'DIVISADEROS',
       'NACOZARI DE GARCÍA', 'ALAMOS', 'HUATABAMPO', 'ETCHOJOA', 'NACO',
       'NAVOJOA', 'CAJEME', 'BÁCUM', 'SAN IGNACIO RÍO MUERTO', 'GUAYMAS',
       'QUIRIEGO', 'ROSARIO', 'EMPALME', 'PUERTO PEÑASCO', 'CABORCA',
       'GENERAL PLUTARCO ELÍAS CALLES', 'MAGDALENA', 'MOCTEZUMA',
       'RAYON ', 'SAN JAVIER', 'VILLA HIDALGO', 'BACADEHUACHI',
       'BAVIACORA', 'CUCURPE', 'HUEPAC', 'MAZATAN', 'NACOZARI DE GARCIA',
       'SAN FELIPE', 'SAN PEDRO DE CUEVA', 'SUAQUI GRANDE',
       'VILLA PESQUEIRA', 'ARIVECHI', 'BACERAC', 'BAVISPE', 'FR

In [25]:
df_water_site_filtered_sonora = df_water_site_filtered[
    (df_water_site_filtered['ESTADO'] == 'SONORA') &
    (df_water_site_filtered['MUNICIPIO'].isin(municipality))    
]
df_water_site_filtered_sonora.head()

,CLAVE SITIO,ESTADO,MUNICIPIO,CUERPO DE AGUA,SUBTIPO CUERPO AGUA,LATITUD,LONGITUD
4697,OCNOR3990,SONORA,ACONCHI,RIO SONORA,RÍO,29.84478,-110.27754
4728,OCNOR4021,SONORA,ACONCHI,ACUÍFERO RIO SONORA,POZO,29.77367,-110.20114
4729,OCNOR4022,SONORA,ACONCHI,RIO SONORA,RÍO,29.82440,-110.23701
4730,OCNOR4023,SONORA,BANÁMICHI,RIO SONORA,RÍO,30.01704,-110.21963
4731,OCNOR4024,SONORA,ARIZPE,RIO SONORA,RÍO,30.33503,-110.16534


In [26]:
df_water_site_filtered_sonora.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25 entries, 4697 to 6491
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CLAVE SITIO          25 non-null     object 
 1   ESTADO               25 non-null     object 
 2   MUNICIPIO            25 non-null     object 
 3   CUERPO DE AGUA       25 non-null     object 
 4   SUBTIPO CUERPO AGUA  25 non-null     object 
 5   LATITUD              25 non-null     float64
 6   LONGITUD             25 non-null     float64
dtypes: float64(2), object(5)
memory usage: 1.6+ KB


In [27]:
# The 'pollutants' list includes the pollutants that will be used in this study
pollutants = ['OD_mg/L', 'DBO_TOT', 'DQO_TOT', 'COLI_FEC', 'E_COLI', 'N_TOT', 'P_TOT', 'TOX_D_48_UT', 'TOX_FIS_SUP_15_UT' ]

In [28]:
body_of_water = df_water_result['TIPO CUERPO DE AGUA'].unique()
body_of_water

# The 'body_of_water' list includes the type of water body you want to study.
body_of_water = []

In [29]:
# Filter 'df_water_result' to only include necesary columns
df_water_result_filtered = df_water_result[['CLAVE SITIO', 'TIPO CUERPO DE AGUA', 'Año'] + pollutants]

# # Filter 'df water_result' so that it does not include any water body type related to 'COSTERO', since this type of water body is not useful for the study
df_water_result_filtered = df_water_result_filtered[~df_water_result_filtered['TIPO CUERPO DE AGUA'].str.contains('COSTERO', na=False)]
print(df_water_result_filtered['TIPO CUERPO DE AGUA'].unique())
df_water_result_filtered.head()

['SUBTERRÁNEO' 'LÓTICO' 'LÉNTICO' 'LÉNTICO (HUMEDAL)' 'LÓTICO (HUMEDAL)'
 'LÓTICO (ESTUDIO ESPECIAL)' 'SUBTERRÁNEO (ESTUDIO ESPECIAL)'
 'LÉNTICO (ESTUDIO ESPECIAL)' 'LENTICO (HUMEDAL)' 'SUBTERRÁNEO (HUMEDAL)'
 'LÓTICO Y SEDIMENTOS']


,CLAVE SITIO,TIPO CUERPO DE AGUA,Año,OD_mg/L,DBO_TOT,DQO_TOT,COLI_FEC,E_COLI,N_TOT,P_TOT,TOX_D_48_UT,TOX_FIS_SUP_15_UT
0,BROTE CARMINA 3,SUBTERRÁNEO,2022,NaN,NaN,NaN,97,NaN,NaN,NaN,NaN,NaN
1,CARMINA 2,LÓTICO,2022,8.13,<2,<10,1076,41,NaN,NaN,<1,NaN
2,CAZEPA-1,SUBTERRÁNEO,2021,5.96,NaN,NaN,<10,NaN,0.612951,0.0381,NaN,NaN
3,CAZEPA-1,SUBTERRÁNEO,2022,NaN,NaN,NaN,<10,NaN,NaN,NaN,NaN,NaN
4,CAZEPA-1,SUBTERRÁNEO,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
df_water_merged = pd.merge(df_water_site_filtered_sonora, df_water_result_filtered, on='CLAVE SITIO', how='inner')
df_water_merged.head(10)

,CLAVE SITIO,ESTADO,MUNICIPIO,CUERPO DE AGUA,SUBTIPO CUERPO AGUA,LATITUD,LONGITUD,TIPO CUERPO DE AGUA,Año,OD_mg/L,DBO_TOT,DQO_TOT,COLI_FEC,E_COLI,N_TOT,P_TOT,TOX_D_48_UT,TOX_FIS_SUP_15_UT
0,OCNOR3990,SONORA,ACONCHI,RIO SONORA,RÍO,29.84478,-110.27754,LÓTICO,2012,<1,104.51,193.8,<3,<3,5.7786,2.0608,<1,NaN
1,OCNOR3990,SONORA,ACONCHI,RIO SONORA,RÍO,29.84478,-110.27754,LÓTICO,2013,NaN,256,1020.572,>24196,305,5.6104,5.0175,NaN,NaN
2,OCNOR3990,SONORA,ACONCHI,RIO SONORA,RÍO,29.84478,-110.27754,LÓTICO,2013,6.21,2.08,26.4,110,52,0.6545,0.055,1.28,NaN
3,OCNOR3990,SONORA,ACONCHI,RIO SONORA,RÍO,29.84478,-110.27754,LÓTICO,2013,7.13,4.01,13.8,1145,160,0.3137,0.0204,<1,NaN
4,OCNOR3990,SONORA,ACONCHI,RIO SONORA,RÍO,29.84478,-110.27754,LÓTICO,2013,7.3,3.09,21.112,2247,<1,0.326,0.034,<1,NaN
5,OCNOR3990,SONORA,ACONCHI,RIO SONORA,RÍO,29.84478,-110.27754,LÓTICO,2014,7.2,2.8,42.61,1674,1470,0.177,0.042,<1,NaN
6,OCNOR3990,SONORA,ACONCHI,RIO SONORA,RÍO,29.84478,-110.27754,LÓTICO,2014,5.9,4.7,74.34,7270,479,0.564,0.04,<1,NaN
7,OCNOR3990,SONORA,ACONCHI,RIO SONORA,RÍO,29.84478,-110.27754,LÓTICO,2014,6.4,21.5,54.64,>24196,110,0.058,0.001,<1,NaN
8,OCNOR3990,SONORA,ACONCHI,RIO SONORA,RÍO,29.84478,-110.27754,LÓTICO,2014,7.7,<2,<10,2851,10,0.299,0.017,<1,NaN
9,OCNOR3990,SONORA,ACONCHI,RIO SONORA,RÍO,29.84478,-110.27754,LÓTICO,2015,8.76,<2,<10,187,<1,0.156198,0.013936,<1,NaN


In [31]:
print(df_water_merged.info())  # Structure summary
print('\n----------------------------------------------------\n')
print('\n----------------------------------------------------\n')
print(df_water_merged.isnull().sum())  # Check for null values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219 entries, 0 to 218
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CLAVE SITIO          219 non-null    object 
 1   ESTADO               219 non-null    object 
 2   MUNICIPIO            219 non-null    object 
 3   CUERPO DE AGUA       219 non-null    object 
 4   SUBTIPO CUERPO AGUA  219 non-null    object 
 5   LATITUD              219 non-null    float64
 6   LONGITUD             219 non-null    float64
 7   TIPO CUERPO DE AGUA  219 non-null    object 
 8   Año                  219 non-null    int64  
 9   OD_mg/L              146 non-null    object 
 10  DBO_TOT              144 non-null    object 
 11  DQO_TOT              144 non-null    object 
 12  COLI_FEC             207 non-null    object 
 13  E_COLI               143 non-null    object 
 14  N_TOT                190 non-null    object 
 15  P_TOT                187 non-null    obj

In [32]:
# Clean the columns of contaminants by removing the > and < symbols and converting them to numeric
for column in pollutants:
    if column in df_water_merged.columns:
        df_water_merged[column] = df_water_merged[column].astype(str)
        df_water_merged[column] = (
            df_water_merged[column]
            .str.replace('<', '', regex=False)
            .str.replace('>', '', regex=False)
        )
        df_water_merged[column] = pd.to_numeric(df_water_merged[column], errors='coerce')

print(df_water_merged.info()) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219 entries, 0 to 218
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CLAVE SITIO          219 non-null    object 
 1   ESTADO               219 non-null    object 
 2   MUNICIPIO            219 non-null    object 
 3   CUERPO DE AGUA       219 non-null    object 
 4   SUBTIPO CUERPO AGUA  219 non-null    object 
 5   LATITUD              219 non-null    float64
 6   LONGITUD             219 non-null    float64
 7   TIPO CUERPO DE AGUA  219 non-null    object 
 8   Año                  219 non-null    int64  
 9   OD_mg/L              146 non-null    float64
 10  DBO_TOT              144 non-null    float64
 11  DQO_TOT              144 non-null    float64
 12  COLI_FEC             207 non-null    float64
 13  E_COLI               143 non-null    float64
 14  N_TOT                190 non-null    float64
 15  P_TOT                187 non-null    flo

In [33]:
INTERIM_WATER_DATA_DIR = INTERIM_DATA_DIR / 'water_interim.parquet'
table = pa.Table.from_pandas(df_water_merged)

with pq.ParquetWriter(INTERIM_WATER_DATA_DIR, table.schema) as writer:
    writer.write_table(table)

In [34]:
df = pd.read_parquet(INTERIM_WATER_DATA_DIR)

print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219 entries, 0 to 218
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CLAVE SITIO          219 non-null    object 
 1   ESTADO               219 non-null    object 
 2   MUNICIPIO            219 non-null    object 
 3   CUERPO DE AGUA       219 non-null    object 
 4   SUBTIPO CUERPO AGUA  219 non-null    object 
 5   LATITUD              219 non-null    float64
 6   LONGITUD             219 non-null    float64
 7   TIPO CUERPO DE AGUA  219 non-null    object 
 8   Año                  219 non-null    int64  
 9   OD_mg/L              146 non-null    float64
 10  DBO_TOT              144 non-null    float64
 11  DQO_TOT              144 non-null    float64
 12  COLI_FEC             207 non-null    float64
 13  E_COLI               143 non-null    float64
 14  N_TOT                190 non-null    float64
 15  P_TOT                187 non-null    flo

In [35]:
df_water_result_filtered['FECHA REALIZACIÓN'] = pd.to_datetime(df_water_result_filtered['FECHA REALIZACIÓN'], format='%m/%d/%Y', errors='coerce')
df_water_result_filtered.head()

KeyError: 'FECHA REALIZACIÓN'